In [51]:
#https://www.avito.ru/rossiya/avtomobili
from lxml import html
import requests
import time
import pandas as pd

headers = {'User-Agent': 'Yandex/1.01.001'}

#удаление символов из строки
def deleteLiteralsFromNumberValues(s):
    if s[0].isdigit():
        return ''.join([i for i in s if i.isdigit() or i in ['.',',']])
    else:
        return s
#удаление символов из строки

def getListOfFeatures(link):
    page = requests.get(link,headers=headers)
    tree = html.fromstring(page.content)

    #цена машины
    price = tree.xpath('//span[@class="price-value-string"]/text()')
    if len(price) > 0 :
        price = price[0]
    else:
        price = '0'
    price = price.replace(u'\xa0',' ').replace(u'\u2009',' ').replace(u'\n',' ').replace(u' ','')
    #цена машины

    #описание машины от пользователя
    description = tree.xpath('//div[@class="item-description-text"]/p/text()') 
    #описание машины от пользователя

    attributes = tree.xpath('//li[@class="item-params-list-item"]/span/text()') 
    attributes = map(lambda s: s.split(":")[0] , attributes)
    values = tree.xpath('//li[@class="item-params-list-item"]/text()')
    values2 = values[1::2]
    values2 = map(lambda s: deleteLiteralsFromNumberValues(s.replace('\n', '').replace(" ","").replace(u'\xa0',' ')), values2)
    zipList = list(zip(attributes, values2))
    zipList.append(('Описание пользователя', ' '.join(description)) )
    zipList.append(('Цена', str(price) ))
    
    return zipList

firstLink = 'https://www.avito.ru/volgograd/avtomobili/geely_emgrand_ec7_2013_877004308'
zL = getListOfFeatures(firstLink)
zL

[('Марка', 'Geely'),
 ('Модель', 'EmgrandEC7'),
 ('Год выпуска', '2013'),
 ('Пробег', '45000'),
 ('Тип кузова', 'хетчбэк'),
 ('Цвет', 'серебряный'),
 ('Объём двигателя', '1.5'),
 ('Коробка передач', 'механика'),
 ('Тип двигателя', 'бензин'),
 ('Привод', 'передний'),
 ('Руль', 'левый'),
 ('Состояние', 'не битый'),
 ('Владельцев по ПТС', '1'),
 ('Мощность двигателя', '98..'),
 ('Описание пользователя',
  'Продается автомобиль Джили Эмгрант, 2013 г.в., состояние автомобиля отличное, вложений не требует, полностью обслужен, все время эксплуатации в одних руках, есть сервисная книжка, два комплекта ключей, комплектация полная -КОНДИЦИОНЕР -АБС -ГУР -ПЭП -кожаный салон -музыка Автомобиль можно оформить в КРЕДИТ на хороших условиях ТОРГ, ОБМЕН ______________________________________________________ осмотр ул. Азизбекова 70, Гарантия юридической чистоты'),
 ('Цена', '360000')]

In [52]:
CarBase = pd.DataFrame(columns=([ a for a,b in zipList ]))
CarBase.loc[0] = [ b for a,b in zipList ]
CarBase

,Марка,Модель,Год выпуска,Пробег,Тип кузова,Цвет,Объём двигателя,Коробка передач,Тип двигателя,Привод,Руль,Состояние,Владельцев по ПТС,Мощность двигателя,Описание пользователя,Цена
0,Geely,EmgrandEC7,2013,45000,хетчбэк,серебряный,1.5,механика,бензин,передний,левый,не битый,1,98..,"Продается автомобиль Джили Эмгрант, 2013 г.в.,...",360000


In [53]:
for i in range(1):
    page = requests.get('https://www.avito.ru/rossiya/avtomobili?p=' + str(i+1))
    tree = html.fromstring(page.content)
    allCarsOnPage = tree.xpath('//a[@class="item-description-title-link"]')
    links = list(map(lambda x: x.get("href"),  allCarsOnPage))
    for l in range(len(links)):
        zL = getListOfFeatures('https://www.avito.ru' + links[l])
        CarBaseTemp = pd.DataFrame(columns=([ a for a,b in zL ]))
        CarBaseTemp.loc[0] = [ b for a,b in zL ]
        CarBase =  CarBase.append(CarBaseTemp, ignore_index=True)
        
CarBase

IndexError: string index out of range

In [16]:
#парсим все ссылки на географические области
mainAvtomobPage = requests.get('https://www.avito.ru/rossiya/transport')
tree = html.fromstring(mainAvtomobPage.content)

#regions = tree.xpath('//div[@class="catalog-counts__more"]/text()')[0]
regions = tree.xpath('//a[@class="js-catalog-counts__link"]')
reg = list(map(lambda x: x.get("href").replace(u"transport","avtomobili") , regions))[5:]
reg

['/moskva/avtomobili',
 '/moskovskaya_oblast/avtomobili',
 '/sankt-peterburg/avtomobili',
 '/leningradskaya_oblast/avtomobili',
 '/bashkortostan/avtomobili',
 '/belgorodskaya_oblast/avtomobili',
 '/volgogradskaya_oblast/avtomobili',
 '/voronezhskaya_oblast/avtomobili',
 '/kaliningradskaya_oblast/avtomobili',
 '/krasnodarskiy_kray/avtomobili',
 '/nizhegorodskaya_oblast/avtomobili',
 '/novosibirskaya_oblast/avtomobili',
 '/omskaya_oblast/avtomobili',
 '/permskiy_kray/avtomobili',
 '/rostovskaya_oblast/avtomobili',
 '/samarskaya_oblast/avtomobili',
 '/saratovskaya_oblast/avtomobili',
 '/sverdlovskaya_oblast/avtomobili',
 '/stavropolskiy_kray/avtomobili',
 '/tatarstan/avtomobili',
 '/tyumenskaya_oblast/avtomobili',
 '/udmurtiya/avtomobili',
 '/chelyabinskaya_oblast/avtomobili',
 '/yaroslavskaya_oblast/avtomobili']

In [79]:
allMachines = requests.get('https://www.avito.ru/volgogradskaya_oblast/avtomobili')
tree = html.fromstring(allMachines.content)

regions = tree.xpath('//a[@class="item-description-title-link"]')
ak = list(map(lambda x: x.get("href"), regions))
last_page = tree.xpath('//a[@class="pagination-page"]')[-1]
last_page = last_page.get("href")
numOfIterationScrapping = int(last_page.split("=")[-1])
numOfIterationScrapping

266

In [21]:
#/volgogradskaya_oblast/avtomobili?p=2

CarBase = pd.DataFrame(columns=([ a for a,b in zipList ]))
CarBase.loc[0] = [ b for a,b in zipList ]

In [22]:
CarBase

,Марка,Модель,Год выпуска,Пробег,Тип кузова,Цвет,Объём двигателя,Коробка передач,Тип двигателя,Привод,Руль,Состояние,Владельцев по ПТС,Мощность двигателя,Описание пользователя,Цена
0,Geely,EmgrandEC7,2013,45000,хетчбэк,серебряный,1.5,механика,бензин,передний,левый,не битый,1,98..,"Продается автомобиль Джили Эмгрант, 2013 г.в.,...",360000


In [9]:
str(5)

'5'

In [23]:
page = requests.get('https://www.avito.ru/yoshkar-ola/avtomobili/volkswagen_touareg_2012_827824885',headers=headers)
tree = html.fromstring(page.content)

#удаление символов из строки
def deleteLiteralsFromNumberValues(s):
    if s[0].isdigit():
        return ''.join([i for i in s if i.isdigit() or i in ['.',',']])
    else:
        return s
#удаление символов из строки

#цена машины
price = tree.xpath('//span[@class="price-value-string"]/text()')[0]
price = price.replace(u'\xa0',' ').replace(u'\u2009',' ').replace(u'\n',' ').replace(u' ','')
#цена машины

#описание машины от пользователя
description = tree.xpath('//div[@class="item-description-text"]/p/text()') 
#описание машины от пользователя

attributes = tree.xpath('//li[@class="item-params-list-item"]/span/text()') 
attributes = map(lambda s: s.split(":")[0] , attributes)
values = tree.xpath('//li[@class="item-params-list-item"]/text()')
values2 = values[1::2]
values2 = map(lambda s: deleteLiteralsFromNumberValues(s.replace('\n', '').replace(" ","").replace(u'\xa0',' ')), values2)
zipList = list(zip(attributes, values2))
zipList.append(('Описание пользователя', ' '.join(description)) )
zipList.append(('Цена', str(price) ))
zipList

[('Марка', 'Volkswagen'),
 ('Модель', 'Touareg'),
 ('Год выпуска', '2012'),
 ('Пробег', '70000'),
 ('Тип кузова', 'внедорожник'),
 ('Цвет', 'белый'),
 ('Объём двигателя', '3.6'),
 ('Количество дверей', '5'),
 ('Коробка передач', 'автомат'),
 ('Тип двигателя', 'бензин'),
 ('Привод', 'полный'),
 ('Руль', 'левый'),
 ('Состояние', 'не битый'),
 ('Владельцев по ПТС', '1'),
 ('Мощность двигателя', '249..'),
 ('Описание пользователя',
  'Не бит, не крашен. Один хозяин. Обслуживался у ОД (вся история в сервисной книжке). Пневмоподвеска. Зимняя, летняя резина на дисках. Бампер, капот в пленке. Продажа с номерами.'),
 ('Цена', '1770000')]

In [24]:
CarBase2 = pd.DataFrame(columns=([ a for a,b in zipList ]))
CarBase2.loc[0] = [ b for a,b in zipList ]

In [25]:
CarBase2

,Марка,Модель,Год выпуска,Пробег,Тип кузова,Цвет,Объём двигателя,Количество дверей,Коробка передач,Тип двигателя,Привод,Руль,Состояние,Владельцев по ПТС,Мощность двигателя,Описание пользователя,Цена
0,Volkswagen,Touareg,2012,70000,внедорожник,белый,3.6,5,автомат,бензин,полный,левый,не битый,1,249..,"Не бит, не крашен. Один хозяин. Обслуживался у...",1770000


In [26]:
CarBase

,Марка,Модель,Год выпуска,Пробег,Тип кузова,Цвет,Объём двигателя,Коробка передач,Тип двигателя,Привод,Руль,Состояние,Владельцев по ПТС,Мощность двигателя,Описание пользователя,Цена
0,Geely,EmgrandEC7,2013,45000,хетчбэк,серебряный,1.5,механика,бензин,передний,левый,не битый,1,98..,"Продается автомобиль Джили Эмгрант, 2013 г.в.,...",360000


In [27]:
result =  CarBase.append(CarBase2, ignore_index=True)

In [28]:
result

,Владельцев по ПТС,Год выпуска,Количество дверей,Коробка передач,Марка,Модель,Мощность двигателя,Объём двигателя,Описание пользователя,Привод,Пробег,Руль,Состояние,Тип двигателя,Тип кузова,Цвет,Цена
0,1,2013,NaN,механика,Geely,EmgrandEC7,98..,1.5,"Продается автомобиль Джили Эмгрант, 2013 г.в.,...",передний,45000,левый,не битый,бензин,хетчбэк,серебряный,360000
1,1,2012,5,автомат,Volkswagen,Touareg,249..,3.6,"Не бит, не крашен. Один хозяин. Обслуживался у...",полный,70000,левый,не битый,бензин,внедорожник,белый,1770000


In [29]:
len([0])

1